In [1]:
# ==============================================================================
# FINAL SUBMISSION SCRIPT: Ariel Data Challenge 2025 (Corrected Version)
#
# Methodology:
# 1. Maximal Feature Engineering: Creates a rich feature set from time-series
#    data (statistical, fourier, wavelet) and metadata (polynomial interactions).
# 2. Quantile Regression: Trains three separate XGBoost models to predict the
#    5th, 50th, and 95th percentiles of the target spectrum.
# 3. Multi-Output Handling: Uses scikit-learn's MultiOutputRegressor to handle
#    the 283-dimensional output for the quantile models.
# 4. Uncertainty Calibration: Finds optimal scaling and additive factors on a
#    validation set to calibrate the predicted uncertainty (sigma), using the
#    official competition metric for reliable evaluation.
# 5. Two-Stage Execution (Corrected Logic):
#    - If run in an environment WITHOUT the test set (e.g., interactive editor),
#      it trains the models, finds calibration factors, retrains on all data,
#      and saves everything.
#    - If run in the submission environment (test set is present), it LOADS
#      the saved models and factors, and generates the final submission.csv.
# ==============================================================================

import time
import pickle
import numpy as np
import pandas as pd
import polars as pl
import functools

import scipy.stats
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.multioutput import MultiOutputRegressor
import pywt
import os
from tqdm import tqdm
import multiprocessing

class Config:
    """Contains all hyperparameters and file paths in one place."""

    # Set paths for Kaggle environment
    DATA_PATH = '/kaggle/input/ariel-data-challenge-2025/'
    PREPROCESSED_PATH = '/kaggle/input/ariel-data-challenge-2025-af-npy/'
    OUTPUT_PATH = '/kaggle/input/ariel-data-2025-27' # Models and artifacts will be saved here

    TRAIN_LABELS_PATH = os.path.join(DATA_PATH, 'train.csv')
    TRAIN_STAR_INFO_PATH = os.path.join(DATA_PATH, 'train_star_info.csv')
    TEST_STAR_INFO_PATH = os.path.join(DATA_PATH, 'test_star_info.csv')
    SAMPLE_SUBMISSION_PATH = os.path.join(DATA_PATH, 'sample_submission.csv')
    WAVELENGTHS_PATH = os.path.join(DATA_PATH, 'wavelengths.csv')
    
    # Using pre-processed raw data for faster execution.
    # In a real run, you might generate these from scratch.
    A_RAW_PATH = os.path.join(PREPROCESSED_PATH, "a_raw_train.npy")
    F_RAW_PATH = os.path.join(PREPROCESSED_PATH, "f_raw_train.npy")

    # Modeling 
    VALIDATION_SPLIT = 0.1
    RANDOM_STATE = 42
    QUANTILES = [0.05, 0.50, 0.95]
    XGB_PARAMS = {
        'n_estimators': 400,
        'learning_rate': 0.04,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.7,
        'random_state': RANDOM_STATE,
        'tree_method': 'hist',
    }
    
    # Calibration Search Space 
    CALIBRATION_SCALING_FACTORS = [1.0, 1.2, 1.5, 2.0, 2.5]
    CALIBRATION_ADDITIVE_FACTORS = [0.0, 0.0005, 0.001, 0.0015]

config = Config()

def f_read_and_preprocess(dataset, planet_ids):
    """Read the FGS1 files for all planet_ids and extract the time series."""
    print(f"Preprocessing FGS1 data for {dataset} set...")
    f_raw_data = np.full((len(planet_ids), 67500), np.nan, dtype=np.float32)
    for i, planet_id in tqdm(list(enumerate(planet_ids)), desc="FGS1"):
        path = f'/kaggle/input/ariel-data-challenge-2025/{dataset}/{int(planet_id)}/FGS1_signal_0.parquet'
        f_signal = pl.read_parquet(path)
        mean_signal = f_signal.cast(pl.Int32).sum_horizontal().cast(pl.Float32).to_numpy() / 1024
        net_signal = mean_signal[1::2] - mean_signal[0::2]
        f_raw_data[i] = net_signal
    return f_raw_data

def a_read_and_preprocess(dataset, planet_ids):
    """Read the AIRS-CH0 files for all planet_ids and extract the time series."""
    print(f"Preprocessing AIRS-CH0 data for {dataset} set...")
    a_raw_data = np.full((len(planet_ids), 5625), np.nan, dtype=np.float32)
    for i, planet_id in tqdm(list(enumerate(planet_ids)), desc="AIRS-CH0"):
        path = f'/kaggle/input/ariel-data-challenge-2025/{dataset}/{int(planet_id)}/AIRS-CH0_signal_0.parquet'
        signal = pl.read_parquet(path)
        mean_signal = signal.cast(pl.Int32).sum_horizontal().cast(pl.Float32).to_numpy() / (32*356)
        net_signal = mean_signal[1::2] - mean_signal[0::2]
        a_raw_data[i] = net_signal
    return a_raw_data

# ==============================================================================
# CORRECTED SCORING FUNCTION
# This function now correctly implements the official weighted GLL metric.
# ==============================================================================
def official_competition_score(y_true, y_pred, sigma_pred, naive_mean, naive_sigma,
                               fsg_sigma_true=1e-6, airs_sigma_true=1e-5, fgs_weight=2.0):
    """
    Calculates the score based on the official weighted Gaussian Log-Likelihood metric.
    """
    y_true, y_pred, sigma_pred = np.array(y_true), np.array(y_pred), np.array(sigma_pred)
    
    # Clip sigma to prevent log(0) or division by zero
    sigma_pred = np.clip(sigma_pred, 1e-15, None)

    # Define ground truth sigma per channel (1 for FGS1, 282 for AIRS)
    sigma_true_per_channel = np.append(np.array([fsg_sigma_true]), np.ones(y_true.shape[1] - 1) * airs_sigma_true)
    sigma_true = np.tile(sigma_true_per_channel, (y_true.shape[0], 1))

    # Calculate GLLs element-wise for each data point
    GLL_pred = scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred)
    GLL_true = scipy.stats.norm.logpdf(y_true, loc=y_true, scale=sigma_true)
    GLL_mean = scipy.stats.norm.logpdf(y_true, loc=naive_mean, scale=naive_sigma)
    
    # Calculate individual scores element-wise, adding epsilon for stability
    denominator = GLL_true - GLL_mean
    ind_scores = (GLL_pred - GLL_mean) / (denominator + 1e-9)

    # Define weights per channel (higher weight for FGS1)
    weights_per_channel = np.append(np.array([fgs_weight]), np.ones(y_true.shape[1] - 1))
    weights = np.tile(weights_per_channel, (y_true.shape[0], 1))

    # Calculate the final score as a weighted average of individual scores
    final_score = np.average(ind_scores, weights=weights)
    
    return float(np.clip(final_score, 0.0, 1.0))

# FEATURE ENGINEERING
def maximal_feature_engineering(f_raw, a_raw, star_info_df):
    """Creates the final, maximal feature set for the models."""
    print("Engineering MAXIMAL features...")
    
    # --- Base Time Windows & Unobscured calculations ---
    fgs_pre = f_raw[:, :20500]; fgs_post = f_raw[:, 47000:]
    fgs_unobscured_mean = (fgs_pre.mean(axis=1) + fgs_post.mean(axis=1)) / 2
    fgs_unobscured_std = (fgs_pre.std(axis=1) + fgs_post.std(axis=1)) / 2
    fgs_transit = f_raw[:, 23500:44000]
    
    # --- Time-series Features ---
    features = {}
    for i in range(5):
        f_slice_mean = fgs_transit[:, i*4100:(i+1)*4100].mean(axis=1)
        features[f'fgs_slice_{i+1}'] = (fgs_unobscured_mean - f_slice_mean) / fgs_unobscured_mean
    
    features['fgs_transit_std'] = fgs_transit.std(axis=1)
    features['fgs_transit_skew'] = scipy.stats.skew(fgs_transit, axis=1)
    features['fgs_transit_kurtosis'] = scipy.stats.kurtosis(fgs_transit, axis=1)
    features['fgs_snr'] = (fgs_unobscured_mean - fgs_transit.mean(axis=1)) / fgs_unobscured_std
    features_df = pd.DataFrame(features, index=star_info_df.index)

    # --- Fourier & Wavelet Features ---
    fft_coeffs = np.fft.fft(fgs_transit, axis=1)
    for i in range(1, 6):
        features_df[f'fgs_fft_mag_{i}'] = np.abs(fft_coeffs[:, i])
    for level in range(1, 4):
        coeffs = pywt.wavedec(fgs_transit, 'db4', level=level, axis=1)
        features_df[f'fgs_wavelet_std_level{level}'] = np.std(coeffs[0], axis=1)
        features_df[f'fgs_wavelet_mean_level{level}'] = np.mean(coeffs[0], axis=1)

    # --- Metadata & Polynomial Features ---
    meta_df = star_info_df.copy().fillna(star_info_df.median())
    meta_df['log_g_proxy'] = np.log1p(meta_df['Ms']) - 2 * np.log1p(meta_df['Rs'])
    meta_df['rho_star_proxy'] = meta_df['Ms'] / (meta_df['Rs']**3)
    
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
    poly_cols = ['Rs', 'Ts', 'Mp', 'P']
    poly_features = poly.fit_transform(meta_df[poly_cols])
    poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(poly_cols), index=meta_df.index)
    
    final_features_df = pd.concat([features_df, meta_df, poly_df], axis=1)
    final_features_df = final_features_df.loc[:, ~final_features_df.columns.duplicated()]

    print(f"Created {final_features_df.shape[1]} features in total.")
    return final_features_df.fillna(0)


if __name__ == '__main__':
    
    # ==========================================================================
    # CORE LOGIC CORRECTION: The if/else statement is now correctly structured.
    # ==========================================================================
    
    # This check correctly identifies if the script is in submission mode.
    is_submission_run = True # os.path.exists(config.TEST_STAR_INFO_PATH)

    if is_submission_run:
        # --- SUBMISSION MODE ---
        # This block now runs ONLY when submitting. It LOADS pre-trained models.
        # It will not time out.
        
        print("\n" + "="*50)
        print("======         SUBMISSION MODE          ======")
        print("="*50 + "\n")

        # 1. Load test set info
        sample_submission = pd.read_csv(config.SAMPLE_SUBMISSION_PATH, index_col='planet_id')
        test_star_info_df = pd.read_csv(config.TEST_STAR_INFO_PATH, index_col='planet_id')
        wavelengths_df = pd.read_csv(config.WAVELENGTHS_PATH)
        target_column_names = wavelengths_df.columns.tolist()

        # 2. Process test set data and create features
        f_raw_test = f_read_and_preprocess('test', test_star_info_df.index)
        a_raw_test = a_read_and_preprocess('test', test_star_info_df.index)
        test_features_df = maximal_feature_engineering(f_raw_test, a_raw_test, test_star_info_df)
        
        # 3. Load saved artifacts (models, columns, calibration params)
        print("Loading saved models and parameters from /kaggle/working/ ...")
        with open(os.path.join(config.OUTPUT_PATH, 'feature_columns.pkl'), 'rb') as f:
            train_cols = pickle.load(f)
        with open(os.path.join(config.OUTPUT_PATH, 'calibration_params.pkl'), 'rb') as f:
            calibration_params = pickle.load(f)
        
        trained_models = {}
        for q in config.QUANTILES:
            model_path = os.path.join(config.OUTPUT_PATH, f'model_quantile_{q}.pkl')
            print(f"Loading model: {model_path}")
            with open(model_path, 'rb') as f:
                trained_models[q] = pickle.load(f)
        
        # 4. Ensure test feature columns match training columns
        test_features_df = test_features_df[train_cols]

        # 5. Make predictions on the test set
        test_quantile_preds = {}
        for q in config.QUANTILES:
            print(f"Predicting for quantile {q}...")
            test_quantile_preds[q] = trained_models[q].predict(test_features_df)

        # 6. Apply calibration and create final submission arrays
        y_pred_test = test_quantile_preds[0.50].clip(0, None) # Clip mean prediction to be non-negative
        lower_test, upper_test = test_quantile_preds[0.05], test_quantile_preds[0.95]
        
        sigma_raw_test = (upper_test - lower_test) / 3.29 # Corresponds to z-score for 90% confidence
        sigma_raw_test[sigma_raw_test < 0] = 1e-10
        
        sigma_pred_test = (sigma_raw_test * calibration_params['scaling']) + calibration_params['additive']
        
        # 7. Format into submission DataFrame
        print("Creating submission file...")
        pred_df = pd.DataFrame(y_pred_test, index=sample_submission.index, columns=target_column_names)
        sigma_df = pd.DataFrame(sigma_pred_test, index=sample_submission.index, columns=[f"sigma_{i+1}" for i in range(len(target_column_names))])
        submission_df = pd.concat([pred_df, sigma_df], axis=1)
        
        submission_df.to_csv('submission.csv')
        print("\n'submission.csv' created successfully!")
        print("\nSubmission file preview:")
        print(submission_df.head())

    else:
        # --- TRAINING MODE ---
        # This block now runs ONLY when you are in the interactive editor.
        # It TRAINS models and SAVES artifacts to /kaggle/working/
        
        print("\n" + "="*50)
        print("======          TRAINING MODE           ======")
        print("="*50 + "\n")
        
        # 1. Load training data and create features
        train_labels_df = pd.read_csv(config.TRAIN_LABELS_PATH, index_col='planet_id')
        train_star_info_df = pd.read_csv(config.TRAIN_STAR_INFO_PATH, index_col='planet_id').loc[train_labels_df.index]
        
        # For training, we load pre-processed data to save time. 
        # Make sure the 'ariel-data-challenge-2025-af-npy' dataset is added to your notebook.
        print("Loading pre-processed training data...")
        f_raw_train, a_raw_train = np.load(config.F_RAW_PATH), np.load(config.A_RAW_PATH)
        
        train_features_df = maximal_feature_engineering(f_raw_train, a_raw_train, train_star_info_df)
        train_labels = train_labels_df.values
        naive_mu_train, naive_sigma_train = np.mean(train_labels), np.std(train_labels)

        # 2. Perform a validation split to find calibration factors
        X_train, X_val, y_train, y_val = train_test_split(
            train_features_df, train_labels, test_size=config.VALIDATION_SPLIT, random_state=config.RANDOM_STATE
        )
        
        val_quantile_preds = {}
        for q in config.QUANTILES:
            print(f"\n--- [Validation] Training model for quantile: {q} ---")
            model = xgb.XGBRegressor(**config.XGB_PARAMS, objective='reg:quantileerror', quantile_alpha=q)
            wrapper = MultiOutputRegressor(model, n_jobs=-1)
            wrapper.fit(X_train, y_train)
            val_quantile_preds[q] = wrapper.predict(X_val)

        # 3. Find the best calibration factors on the validation set using the CORRECT metric
        y_pred_val, lower_val, upper_val = val_quantile_preds[0.50], val_quantile_preds[0.05], val_quantile_preds[0.95]
        sigma_raw_val = (upper_val - lower_val) / 3.29
        sigma_raw_val[sigma_raw_val < 0] = 1e-10

        print("\n--- [Validation] Searching for best calibration factors using OFFICIAL metric... ---")
        best_score, best_scaling, best_additive = -1.0, 1.0, 0.0
        for scaling in config.CALIBRATION_SCALING_FACTORS:
            for additive in config.CALIBRATION_ADDITIVE_FACTORS:
                sigma_calibrated = (sigma_raw_val * scaling) + additive
                # USE THE CORRECT, OFFICIAL SCORING FUNCTION
                score = official_competition_score(y_val, y_pred_val, sigma_calibrated, naive_mu_train, naive_sigma_train)
                
                print(f"  - Testing Scale={scaling}, Add={additive} -> Score: {score:.4f}")
                if score > best_score:
                    best_score, best_scaling, best_additive = score, scaling, additive
        
        print(f"\n--- Best factors found: Scale={best_scaling}, Add={best_additive} (Official CV Score: {best_score:.4f}) ---")
        
        # 4. Save the determined calibration factors and feature columns
        os.makedirs(config.OUTPUT_PATH, exist_ok=True)
        calibration_params = {'scaling': best_scaling, 'additive': best_additive}
        with open(os.path.join(config.OUTPUT_PATH, 'calibration_params.pkl'), 'wb') as f:
            pickle.dump(calibration_params, f)
        
        with open(os.path.join(config.OUTPUT_PATH, 'feature_columns.pkl'), 'wb') as f:
            pickle.dump(train_features_df.columns.tolist(), f)

        # 5. Retrain final models on ALL available training data and save them
        print("\n--- Retraining models on full dataset for submission... ---")
        for q in config.QUANTILES:
            print(f"\n--- [Full Data] Training model for quantile: {q} ---")
            model = xgb.XGBRegressor(**config.XGB_PARAMS, objective='reg:quantileerror', quantile_alpha=q)
            wrapper = MultiOutputRegressor(model, n_jobs=-1)
            wrapper.fit(train_features_df, train_labels) # Use ALL data
            model_path = os.path.join(config.OUTPUT_PATH, f'model_quantile_{q}.pkl')
            print(f"Saving model to: {model_path}")
            with open(model_path, 'wb') as f:
                pickle.dump(wrapper, f)
        
        print("\nTraining complete. All necessary artifacts saved to /kaggle/working/")


======         SUBMISSION MODE          ======

Preprocessing FGS1 data for test set...


FGS1: 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


Preprocessing AIRS-CH0 data for test set...


AIRS-CH0: 100%|██████████| 1/1 [00:00<00:00,  1.16it/s]


Engineering MAXIMAL features...
Created 36 features in total.
Loading saved models and parameters from /kaggle/working/ ...
Loading model: /kaggle/input/ariel-data-2025-27/model_quantile_0.05.pkl
Loading model: /kaggle/input/ariel-data-2025-27/model_quantile_0.5.pkl
Loading model: /kaggle/input/ariel-data-2025-27/model_quantile_0.95.pkl
Predicting for quantile 0.05...
Predicting for quantile 0.5...
Predicting for quantile 0.95...
Creating submission file...

'submission.csv' created successfully!

Submission file preview:
               wl_1      wl_2      wl_3      wl_4      wl_5      wl_6  \
planet_id                                                               
1103775    0.015889  0.015461  0.015714  0.015221  0.015117  0.015069   

               wl_7      wl_8      wl_9     wl_10  ...  sigma_274  sigma_275  \
planet_id                                          ...                         
1103775    0.015974  0.015672  0.015994  0.015661  ...   0.001499   0.001662   

           